In [376]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns

# Importing packages
%matplotlib inline
import matplotlib.pyplot as plt
import csv
import sklearn
import cPickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.learning_curve import learning_curve

In [377]:
df1 = pd.read_csv('./atasets/WDI - Education.csv')
df1 = df1[['Country Name','Time','Country Code','Adult literacy rate, population 15+ years, both sexes (%) [SE.ADT.LITR.ZS]','All education staff compensation, primary (% of total expenditure in primary public institutions) [SE.XPD.MPRM.ZS]','Children out of school (% of primary school age) [SE.PRM.UNER.ZS]','Current education expenditure, total (% of total expenditure in public institutions) [SE.XPD.CTOT.ZS]','Duration of compulsory education (years) [SE.COM.DURS]','Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative) [SE.PRM.CUAT.FE.ZS]','Enrolment in primary education, both sexes (number) [SE.PRM.ENRL]','Expenditure on education as % of total government expenditure (%) [SE.XPD.TOTL.GB.ZS]','Government expenditure on education, total (% of GDP) [SE.XPD.TOTL.GD.ZS]','Government expenditure per student, primary (% of GDP per capita) [SE.XPD.PRIM.PC.ZS]','Gross enrollment ratio, primary, both sexes (%) [SE.PRM.ENRR]','Literacy rate, adult female (% of females ages 15 and above) [SE.ADT.LITR.FE.ZS]','Literacy rate, adult male (% of males ages 15 and above) [SE.ADT.LITR.MA.ZS]','Net enrolment rate, primary, both sexes (%) [SE.PRM.NENR]','Out-of-school children of primary school age, both sexes (number) [SE.PRM.UNER]','Percentage of enrolment in primary education in private institutions (%) [SE.PRM.PRIV.ZS]','Primary completion rate, total (% of relevant age group) [SE.PRM.CMPT.ZS]','School enrollment, primary (gross), gender parity index (GPI) [SE.ENR.PRIM.FM.ZS]','Teachers in secondary education, both sexes (number) [SE.SEC.TCHR]','Youth literacy rate, population 15-24 years, both sexes (%) [SE.ADT.1524.LT.ZS]']]
df2 = pd.read_csv('./atasets/WDI - Economics - National accounts.csv')
df2 = df2[['Country Name','Time','Country Code','Adjusted net national income (annual % growth) [NY.ADJ.NNTY.KD.ZG]','Adjusted net national income (constant 2010 US$) [NY.ADJ.NNTY.KD]','Adjusted net national income (current US$) [NY.ADJ.NNTY.CD]','Adjusted net national income per capita (annual % growth) [NY.ADJ.NNTY.PC.KD.ZG]','Adjusted net national income per capita (constant 2010 US$) [NY.ADJ.NNTY.PC.KD]','Adjusted net savings, excluding particulate emission damage (% of GNI) [NY.ADJ.SVNX.GN.ZS]','Adjusted net savings, excluding particulate emission damage (current US$) [NY.ADJ.SVNX.CD]','Adjusted net savings, including particulate emission damage (% of GNI) [NY.ADJ.SVNG.GN.ZS]','Adjusted net savings, including particulate emission damage (current US$) [NY.ADJ.SVNG.CD]','Adjusted savings: education expenditure (% of GNI) [NY.ADJ.AEDU.GN.ZS]','Adjusted savings: education expenditure (current US$) [NY.ADJ.AEDU.CD]','Exports of goods and services (constant 2010 US$) [NE.EXP.GNFS.KD]','GDP (constant LCU) [NY.GDP.MKTP.KN]','GDP (current US$) [NY.GDP.MKTP.CD]','GDP per capita (constant 2010 US$) [NY.GDP.PCAP.KD]','General government final consumption expenditure (constant 2010 US$) [NE.CON.GOVT.KD]','GNI (constant 2010 US$) [NY.GNP.MKTP.KD]','GNI per capita (constant 2010 US$) [NY.GNP.PCAP.KD]','Trade (% of GDP) [NE.TRD.GNFS.ZS]']]
# df3 = pd.read_csv('./atasets/WDI - Economics - Balance of payments.csv') not needed
# df4 = pd.read_csv('./atasets/WDI - Economics - Development assistance and PPP.csv')
df5 = pd.read_csv('./atasets/WDI - Economics - External debt.csv')
df5 = df5[['Country Name','Time','Country Code','Total debt service (% of GNI) [DT.TDS.DECT.GN.ZS]']]
df6 = pd.read_csv('./atasets/WDI - Environment.csv')
df6 = df6[['Country Name','Time','Country Code','Urban population (% of total) [SP.URB.TOTL.IN.ZS]']]
# df7 = pd.read_csv('./atasets/WDI - Finance.csv') not needed
df8 = pd.read_csv('./atasets/WDI - Health.csv')
df8 = df8[['Country Name','Time','Country Code','Adults (ages 15+) and children (ages 0-14) newly infected with HIV [SH.HIV.INCD.TL]','Birth rate, crude (per 1,000 people) [SP.DYN.CBRT.IN]','Death rate, crude (per 1,000 people) [SP.DYN.CDRT.IN]','Life expectancy at birth, total (years) [SP.DYN.LE00.IN]','Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]','Population, total [SP.POP.TOTL]']]
df9 = pd.read_csv('./atasets/WDI - Infrastructure.csv')
df9 = df9[['Country Name','Time','Country Code','Scientific and technical journal articles [IP.JRN.ARTC.SC]']]
df10 = pd.read_csv('./atasets/WDI - Public sector and Conflict.csv')
df10 = df10[['Country Name','Time','Country Code','Central government debt, total (% of GDP) [GC.DOD.TOTL.GD.ZS]']]
df11 = pd.read_csv('./atasets/WDI - Social and Labor.csv')
df11 = df11[['Country Name','Time','Country Code','Children in employment, total (% of children ages 7-14) [SL.TLF.0714.ZS]','International migrant stock (% of population) [SM.POP.TOTL.ZS]','Unemployment, total (% of total labor force) (national estimate) [SL.UEM.TOTL.NE.ZS]']]
# df12 = pd.read_csv('./atasets/WDI - Trade and Private sector.csv')

In [378]:
df2.head(3)

,Country Name,Time,Country Code,Adjusted net national income (annual % growth) [NY.ADJ.NNTY.KD.ZG],Adjusted net national income (constant 2010 US$) [NY.ADJ.NNTY.KD],Adjusted net national income (current US$) [NY.ADJ.NNTY.CD],Adjusted net national income per capita (annual % growth) [NY.ADJ.NNTY.PC.KD.ZG],Adjusted net national income per capita (constant 2010 US$) [NY.ADJ.NNTY.PC.KD],"Adjusted net savings, excluding particulate emission damage (% of GNI) [NY.ADJ.SVNX.GN.ZS]","Adjusted net savings, excluding particulate emission damage (current US$) [NY.ADJ.SVNX.CD]",...,Adjusted savings: education expenditure (% of GNI) [NY.ADJ.AEDU.GN.ZS],Adjusted savings: education expenditure (current US$) [NY.ADJ.AEDU.CD],Exports of goods and services (constant 2010 US$) [NE.EXP.GNFS.KD],GDP (constant LCU) [NY.GDP.MKTP.KN],GDP (current US$) [NY.GDP.MKTP.CD],GDP per capita (constant 2010 US$) [NY.GDP.PCAP.KD],General government final consumption expenditure (constant 2010 US$) [NE.CON.GOVT.KD],GNI (constant 2010 US$) [NY.GNP.MKTP.KD],GNI per capita (constant 2010 US$) [NY.GNP.PCAP.KD],Trade (% of GDP) [NE.TRD.GNFS.ZS]
0,Afghanistan,1990,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,1990,ALB,NaN,NaN,1.756198e+09,NaN,NaN,6.298541,1.322520e+08,...,2.800000,5.879230e+07,NaN,3.613260e+11,2.101625e+09,1879.654764,NaN,NaN,NaN,38.065783
2,Algeria,1990,DZA,2.493406,6.220759e+10,4.842934e+10,-0.096158,2400.691516,10.181928,6.104585e+09,...,4.946261,2.965535e+09,3.943033e+10,2.134980e+11,6.204510e+10,3551.128516,6.125649e+09,8.959417e+10,3457.583725,48.380714


In [379]:
df_1 = df1.merge(df2, on=['Country Code', 'Time'], how='left')
df_1.head(3)

,Country Name_x,Time,Country Code,"Adult literacy rate, population 15+ years, both sexes (%) [SE.ADT.LITR.ZS]","All education staff compensation, primary (% of total expenditure in primary public institutions) [SE.XPD.MPRM.ZS]",Children out of school (% of primary school age) [SE.PRM.UNER.ZS],"Current education expenditure, total (% of total expenditure in public institutions) [SE.XPD.CTOT.ZS]",Duration of compulsory education (years) [SE.COM.DURS],"Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative) [SE.PRM.CUAT.FE.ZS]","Enrolment in primary education, both sexes (number) [SE.PRM.ENRL]",...,Adjusted savings: education expenditure (% of GNI) [NY.ADJ.AEDU.GN.ZS],Adjusted savings: education expenditure (current US$) [NY.ADJ.AEDU.CD],Exports of goods and services (constant 2010 US$) [NE.EXP.GNFS.KD],GDP (constant LCU) [NY.GDP.MKTP.KN],GDP (current US$) [NY.GDP.MKTP.CD],GDP per capita (constant 2010 US$) [NY.GDP.PCAP.KD],General government final consumption expenditure (constant 2010 US$) [NE.CON.GOVT.KD],GNI (constant 2010 US$) [NY.GNP.MKTP.KD],GNI per capita (constant 2010 US$) [NY.GNP.PCAP.KD],Trade (% of GDP) [NE.TRD.GNFS.ZS]
0,Afghanistan,1990,AFG,NaN,NaN,NaN,NaN,NaN,NaN,622513.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,1990,ALB,NaN,NaN,NaN,NaN,NaN,NaN,278446.0,...,2.800000,5.879230e+07,NaN,3.613260e+11,2.101625e+09,1879.654764,NaN,NaN,NaN,38.065783
2,Algeria,1990,DZA,NaN,NaN,14.60961,NaN,NaN,NaN,4027612.0,...,4.946261,2.965535e+09,3.943033e+10,2.134980e+11,6.204510e+10,3551.128516,6.125649e+09,8.959417e+10,3457.583725,48.380714


In [380]:
df_1.shape

(7139, 43)

In [381]:
df1.shape

(7133, 23)

In [382]:
df2.shape

(7133, 22)

In [383]:
df_2= df_1.merge(df4, on=['Country Code', 'Time'], how='left')
df_3= df_2.merge(df5, on=['Country Code', 'Time'], how='left')
df_4= df_3.merge(df6, on=['Country Code', 'Time'], how='left')
df_6= df_4.merge(df9, on=['Country Code', 'Time'], how='left')
df_7= df_6.merge(df10, on=['Country Code', 'Time'], how='left')
df= df_7.merge(df11, on=['Country Code', 'Time'], how='left')
df.head(3)

,Country Name_x,Time,Country Code,"Adult literacy rate, population 15+ years, both sexes (%) [SE.ADT.LITR.ZS]","All education staff compensation, primary (% of total expenditure in primary public institutions) [SE.XPD.MPRM.ZS]",Children out of school (% of primary school age) [SE.PRM.UNER.ZS],"Current education expenditure, total (% of total expenditure in public institutions) [SE.XPD.CTOT.ZS]",Duration of compulsory education (years) [SE.COM.DURS],"Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative) [SE.PRM.CUAT.FE.ZS]","Enrolment in primary education, both sexes (number) [SE.PRM.ENRL]",...,Country Name_x,Urban population (% of total) [SP.URB.TOTL.IN.ZS],Country Name_y,Scientific and technical journal articles [IP.JRN.ARTC.SC],Country Name_x,"Central government debt, total (% of GDP) [GC.DOD.TOTL.GD.ZS]",Country Name_y,"Children in employment, total (% of children ages 7-14) [SL.TLF.0714.ZS]",International migrant stock (% of population) [SM.POP.TOTL.ZS],"Unemployment, total (% of total labor force) (national estimate) [SL.UEM.TOTL.NE.ZS]"
0,Afghanistan,1990,AFG,NaN,NaN,NaN,NaN,NaN,NaN,622513.0,...,Afghanistan,18.316,NaN,NaN,NaN,NaN,Afghanistan,NaN,0.478025,NaN
1,Albania,1990,ALB,NaN,NaN,NaN,NaN,NaN,NaN,278446.0,...,Albania,36.428,NaN,NaN,NaN,NaN,Albania,NaN,2.011700,NaN
2,Algeria,1990,DZA,NaN,NaN,14.60961,NaN,NaN,NaN,4027612.0,...,Algeria,52.085,NaN,NaN,NaN,NaN,Algeria,NaN,1.057233,19.799999


In [384]:
df.isnull().sum(axis=0)

Country Name_x                                                                                                            6563
Time                                                                                                                      6561
Country Code                                                                                                              6563
Adult literacy rate, population 15+ years, both sexes (%) [SE.ADT.LITR.ZS]                                               12838
All education staff compensation, primary (% of total expenditure in primary public institutions) [SE.XPD.MPRM.ZS]       12564
Children out of school (% of primary school age) [SE.PRM.UNER.ZS]                                                         9967
Current education expenditure, total (% of total expenditure in public institutions) [SE.XPD.CTOT.ZS]                    12502
Duration of compulsory education (years) [SE.COM.DURS]                                                         

In [385]:
 def missing_values_0(data):
    if data.isnull().sum() > 0:
        print "Number of missing values replaced with zeros = ",  data.isnull().sum()
        ''' here you can replace it with the zeros'''
        data.fillna(0, inplace=True)
    elif data.isnull().sum() == 0:
        print "Number of missing values =  0"


In [386]:
 def missing_values_mean(data):
    if data.isnull().sum() > 0:
        print "Number of missing values replaced with mean = ",  data.isnull().sum()
        ''' here you can replace it with the zeros'''
        data.fillna(np.mean(data), inplace=True)
    elif data.isnull().sum() == 0:
        print "Number of missing values =  0"
# need mean by country, will work on this

In [387]:
 def missing_values_drop(data):
    if data.isnull().sum() > 0:
        print "Number of missing values replaced with zeros = ",  data.isnull().sum()
        ''' here you can replace it with the zeros'''
        data.dropna(inplace=True)
    elif data.isnull().sum() == 0:
        print "Number of missing values =  0"

In [388]:
df.drop(['Country Name_y', 'Country Name_x'
          ], axis=1, inplace=True) 

In [389]:
df.isnull().sum(axis=0)

Time                                                                                                                      6561
Country Code                                                                                                              6563
Adult literacy rate, population 15+ years, both sexes (%) [SE.ADT.LITR.ZS]                                               12838
All education staff compensation, primary (% of total expenditure in primary public institutions) [SE.XPD.MPRM.ZS]       12564
Children out of school (% of primary school age) [SE.PRM.UNER.ZS]                                                         9967
Current education expenditure, total (% of total expenditure in public institutions) [SE.XPD.CTOT.ZS]                    12502
Duration of compulsory education (years) [SE.COM.DURS]                                                                   10232
Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative) [SE.PRM.CUAT.

In [390]:
df = df.fillna(df.bfill())

In [391]:
missing_values_mean(df['Adult literacy rate, population 15+ years, both sexes (%) [SE.ADT.LITR.ZS]'])
missing_values_mean(df['All education staff compensation, primary (% of total expenditure in primary public institutions) [SE.XPD.MPRM.ZS]'])
missing_values_mean(df['Children out of school (% of primary school age) [SE.PRM.UNER.ZS]'])
missing_values_mean(df['Current education expenditure, total (% of total expenditure in public institutions) [SE.XPD.CTOT.ZS]'])
missing_values_mean(df['Duration of compulsory education (years) [SE.COM.DURS]'])
missing_values_mean(df['Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative) [SE.PRM.CUAT.FE.ZS]'])
missing_values_mean(df['Enrolment in primary education, both sexes (number) [SE.PRM.ENRL]'])
missing_values_mean(df['Expenditure on education as % of total government expenditure (%) [SE.XPD.TOTL.GB.ZS]'])
missing_values_mean(df['Government expenditure on education, total (% of GDP) [SE.XPD.TOTL.GD.ZS]'])
missing_values_mean(df['Gross enrollment ratio, primary, both sexes (%) [SE.PRM.ENRR]'])
missing_values_mean(df['Government expenditure per student, primary (% of GDP per capita) [SE.XPD.PRIM.PC.ZS]'])
missing_values_mean(df['Literacy rate, adult female (% of females ages 15 and above) [SE.ADT.LITR.FE.ZS]'])
missing_values_mean(df['Literacy rate, adult male (% of males ages 15 and above) [SE.ADT.LITR.MA.ZS]'])
missing_values_mean(df['Net enrolment rate, primary, both sexes (%) [SE.PRM.NENR]'])
missing_values_mean(df['Out-of-school children of primary school age, both sexes (number) [SE.PRM.UNER]'])
missing_values_mean(df['Percentage of enrolment in primary education in private institutions (%) [SE.PRM.PRIV.ZS]'])
missing_values_mean(df['Primary completion rate, total (% of relevant age group) [SE.PRM.CMPT.ZS]'])
missing_values_mean(df['School enrollment, primary (gross), gender parity index (GPI) [SE.ENR.PRIM.FM.ZS]'])
missing_values_mean(df['Teachers in secondary education, both sexes (number) [SE.SEC.TCHR]'])
missing_values_mean(df['Youth literacy rate, population 15-24 years, both sexes (%) [SE.ADT.1524.LT.ZS]'])
missing_values_mean(df['Adjusted net national income (annual % growth) [NY.ADJ.NNTY.KD.ZG]'])
missing_values_mean(df['Adjusted net national income (constant 2010 US$) [NY.ADJ.NNTY.KD]'])
missing_values_mean(df['Adjusted net national income (current US$) [NY.ADJ.NNTY.CD]'])
missing_values_mean(df['Adjusted net national income per capita (annual % growth) [NY.ADJ.NNTY.PC.KD.ZG]'])
missing_values_mean(df['Adjusted net national income per capita (constant 2010 US$) [NY.ADJ.NNTY.PC.KD]'])
missing_values_mean(df['Adjusted net savings, excluding particulate emission damage (% of GNI) [NY.ADJ.SVNX.GN.ZS]'])
missing_values_mean(df['Adjusted net savings, excluding particulate emission damage (current US$) [NY.ADJ.SVNX.CD]'])
missing_values_mean(df['Adjusted net savings, including particulate emission damage (% of GNI) [NY.ADJ.SVNG.GN.ZS]'])
missing_values_mean(df['Adjusted net savings, including particulate emission damage (current US$) [NY.ADJ.SVNG.CD]'])
missing_values_mean(df['Adjusted savings: education expenditure (% of GNI) [NY.ADJ.AEDU.GN.ZS]'])
missing_values_mean(df['Adjusted savings: education expenditure (current US$) [NY.ADJ.AEDU.CD]'])
missing_values_mean(df['Exports of goods and services (constant 2010 US$) [NE.EXP.GNFS.KD]'])
missing_values_mean(df['Urban population (% of total) [SP.URB.TOTL.IN.ZS]'])
missing_values_mean(df['GDP (constant LCU) [NY.GDP.MKTP.KN]'])
missing_values_mean(df['GDP (current US$) [NY.GDP.MKTP.CD]'])
missing_values_mean(df['GDP per capita (constant 2010 US$) [NY.GDP.PCAP.KD]'])
missing_values_mean(df['General government final consumption expenditure (constant 2010 US$) [NE.CON.GOVT.KD]'])
missing_values_mean(df['GNI (constant 2010 US$) [NY.GNP.MKTP.KD]'])
missing_values_mean(df['GNI per capita (constant 2010 US$) [NY.GNP.PCAP.KD]'])
missing_values_mean(df['Trade (% of GDP) [NE.TRD.GNFS.ZS]'])
missing_values_mean(df['Unemployment, total (% of total labor force) (national estimate) [SL.UEM.TOTL.NE.ZS]'])
missing_values_mean(df['International migrant stock (% of population) [SM.POP.TOTL.ZS]'])
missing_values_mean(df['Children in employment, total (% of children ages 7-14) [SL.TLF.0714.ZS]'])
missing_values_mean(df['Central government debt, total (% of GDP) [GC.DOD.TOTL.GD.ZS]'])
missing_values_mean(df['Scientific and technical journal articles [IP.JRN.ARTC.SC]'])
missing_values_mean(df['Total debt service (% of GNI) [DT.TDS.DECT.GN.ZS]'])

Number of missing values replaced with mean =  6827
Number of missing values replaced with mean =  7115
Number of missing values replaced with mean =  6855
Number of missing values replaced with mean =  6845
Number of missing values replaced with mean =  7091
Number of missing values replaced with mean =  6940
Number of missing values replaced with mean =  6845
Number of missing values replaced with mean =  7099
Number of missing values replaced with mean =  6845
Number of missing values replaced with mean =  6855
Number of missing values replaced with mean =  7115
Number of missing values replaced with mean =  6827
Number of missing values replaced with mean =  6827
Number of missing values replaced with mean =  6855
Number of missing values replaced with mean =  6855
Number of missing values replaced with mean =  6855
Number of missing values replaced with mean =  6855
Number of missing values replaced with mean =  6855
Number of missing values replaced with mean =  6845
Number of mi

In [392]:
df.isnull().sum(axis=0)

Time                                                                                                                        0
Country Code                                                                                                             6563
Adult literacy rate, population 15+ years, both sexes (%) [SE.ADT.LITR.ZS]                                                  0
All education staff compensation, primary (% of total expenditure in primary public institutions) [SE.XPD.MPRM.ZS]          0
Children out of school (% of primary school age) [SE.PRM.UNER.ZS]                                                           0
Current education expenditure, total (% of total expenditure in public institutions) [SE.XPD.CTOT.ZS]                       0
Duration of compulsory education (years) [SE.COM.DURS]                                                                      0
Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative) [SE.PRM.CUAT.FE.ZS] 

In [393]:
# dropping redundant variables
df.drop(['GNI (constant 2010 US$) [NY.GNP.MKTP.KD]', 'GDP (current US$) [NY.GDP.MKTP.CD]', 'GDP (constant LCU) [NY.GDP.MKTP.KN]', 'Current education expenditure, total (% of total expenditure in public institutions) [SE.XPD.CTOT.ZS]', 'Expenditure on education as % of total government expenditure (%) [SE.XPD.TOTL.GB.ZS]', 'Literacy rate, adult male (% of males ages 15 and above) [SE.ADT.LITR.MA.ZS]','Adjusted savings: education expenditure (current US$) [NY.ADJ.AEDU.CD]','Adjusted net savings, including particulate emission damage (current US$) [NY.ADJ.SVNG.CD]','Adjusted net savings, excluding particulate emission damage (current US$) [NY.ADJ.SVNX.CD]','Adjusted net national income per capita (constant 2010 US$) [NY.ADJ.NNTY.PC.KD]','Adjusted net national income (current US$) [NY.ADJ.NNTY.CD]','Adjusted net national income (constant 2010 US$) [NY.ADJ.NNTY.KD]' 
          ], axis=1, inplace=True) 

In [394]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
"Adult literacy rate, population 15+ years, both sexes (%) [SE.ADT.LITR.ZS]",13691.0,7.623481e+01,1.616690e+01,1.089465e+01,7.623481e+01,7.623481e+01,8.420696e+01,9.999924e+01
"All education staff compensation, primary (% of total expenditure in primary public institutions) [SE.XPD.MPRM.ZS]",13691.0,7.894001e+01,8.733524e+00,0.000000e+00,7.894001e+01,7.894001e+01,8.022267e+01,1.000000e+02
Children out of school (% of primary school age) [SE.PRM.UNER.ZS],13691.0,1.297626e+01,1.087328e+01,0.000000e+00,6.313040e+00,1.297626e+01,1.297626e+01,8.084902e+01
Duration of compulsory education (years) [SE.COM.DURS],13691.0,8.044091e+00,1.544511e+00,0.000000e+00,8.000000e+00,8.044091e+00,8.044091e+00,1.600000e+01
"Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative) [SE.PRM.CUAT.FE.ZS]",13691.0,7.597877e+01,1.858494e+01,2.561130e+00,7.597877e+01,7.597877e+01,8.550000e+01,1.000000e+02
"Enrolment in primary education, both sexes (number) [SE.PRM.ENRL]",13691.0,2.953048e+07,6.431752e+07,1.134000e+03,9.529525e+05,2.953048e+07,2.953048e+07,7.190548e+08
"Government expenditure on education, total (% of GDP) [SE.XPD.TOTL.GD.ZS]",13691.0,4.331833e+00,1.223317e+00,0.000000e+00,4.148290e+00,4.331833e+00,4.331833e+00,4.433398e+01
"Government expenditure per student, primary (% of GDP per capita) [SE.XPD.PRIM.PC.ZS]",13691.0,1.396002e+01,4.007235e+00,2.846470e+00,1.123864e+01,1.396002e+01,1.396002e+01,5.806676e+01
"Gross enrollment ratio, primary, both sexes (%) [SE.PRM.ENRR]",13691.0,9.966456e+01,1.175086e+01,2.029839e+01,9.966456e+01,9.966456e+01,1.016654e+02,1.752442e+02
"Literacy rate, adult female (% of females ages 15 and above) [SE.ADT.LITR.FE.ZS]",13691.0,7.158196e+01,1.883903e+01,4.591830e+00,7.158196e+01,7.158196e+01,8.165875e+01,9.999903e+01


In [395]:
df.rename(columns={
        'Country Name_x':'country_name',
    }, inplace=True)

In [396]:
df.head(3)

,Time,Country Code,"Adult literacy rate, population 15+ years, both sexes (%) [SE.ADT.LITR.ZS]","All education staff compensation, primary (% of total expenditure in primary public institutions) [SE.XPD.MPRM.ZS]",Children out of school (% of primary school age) [SE.PRM.UNER.ZS],Duration of compulsory education (years) [SE.COM.DURS],"Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative) [SE.PRM.CUAT.FE.ZS]","Enrolment in primary education, both sexes (number) [SE.PRM.ENRL]","Government expenditure on education, total (% of GDP) [SE.XPD.TOTL.GD.ZS]","Government expenditure per student, primary (% of GDP per capita) [SE.XPD.PRIM.PC.ZS]",...,General government final consumption expenditure (constant 2010 US$) [NE.CON.GOVT.KD],GNI per capita (constant 2010 US$) [NY.GNP.PCAP.KD],Trade (% of GDP) [NE.TRD.GNFS.ZS],Total debt service (% of GNI) [DT.TDS.DECT.GN.ZS],Urban population (% of total) [SP.URB.TOTL.IN.ZS],Scientific and technical journal articles [IP.JRN.ARTC.SC],"Central government debt, total (% of GDP) [GC.DOD.TOTL.GD.ZS]","Children in employment, total (% of children ages 7-14) [SL.TLF.0714.ZS]",International migrant stock (% of population) [SM.POP.TOTL.ZS],"Unemployment, total (% of total labor force) (national estimate) [SL.UEM.TOTL.NE.ZS]"
0,1990,AFG,54.593472,89.570389,14.60961,6.0,85.5,622513.0,1.06738,11.00508,...,6.125649e+09,3457.583725,38.065783,14.691752,18.316,1.3,3.67533,48.3,0.478025,19.799999
1,1990,ALB,54.593472,89.570389,14.60961,6.0,85.5,278446.0,1.06738,11.00508,...,6.125649e+09,3457.583725,38.065783,14.691752,36.428,1.3,3.67533,48.3,2.011700,19.799999
2,1990,DZA,54.593472,89.570389,14.60961,6.0,85.5,4027612.0,1.06738,11.00508,...,6.125649e+09,3457.583725,48.380714,14.691752,52.085,1.3,3.67533,48.3,1.057233,19.799999
